## Topic 1.1. Astrometry: measuring positions

In [ ]:
import pandas as pd
import numpy as np

import sys
import math

sys.path.append('..')
import astroutils as astro

### Question 1.3

Converting degrees, arcminutes and arcseconds to radians and vice versa

In [2]:
astro.to_radians(sf = 6, degs=35, arcmins=44, arcsecs=36)

'0.623839'

In [3]:
astro.from_radians(sf=3,radians=0.623839)

('35', '44', '36')

### Question 1.4

Small angle approximation.

In [4]:
astro.to_radians(sf=4, arcsecs=30)

'0.0001454'

In [5]:
astro.to_sf(
    sf=4,
    res=math.sin(float(astro.to_radians(sf=4,arcsecs=30)))
)

'0.0001454'

### Question 1.5

Conversion of RA time coordinates to decimal degrees.

In [6]:
astro.ra_from_time_to_deg(sf=5, h=11, m=45, s=56.78)

'176.49'

### Example 1.4

Calculate the angular separation between `cyg_A_61` and `cyg_B_61` with positions:

`21h 06m 53.94s, +38deg 44arcmin 57.9arcsec`   
`21h 06m 55.26s, +38deg 44arcmin 31.4arcsec` 

respectively. Express your answer in arcseconds.


#### 1. Define locations

In [7]:
cyg_A_61_ra = np.array([21, 6, 53.94])
cyg_A_61_dec = np.array([38, 44, 57.9])
cyg_B_61_ra = np.array([21, 6, 55.26])
cyg_B_61_dec = np.array([38, 44, 31.4])

#### 2. Compute deltas.

In [8]:

delta_ra = cyg_A_61_ra - cyg_B_61_ra
delta_dec = cyg_A_61_dec - cyg_B_61_dec


In [9]:
delta_ra

array([ 0.  ,  0.  , -1.32])

In [10]:
delta_dec

array([ 0. ,  0. , 26.5])

#### 3. Convert RA delta to arcseconds.

In [11]:
delta_ra_arcseconds = delta_ra * 15

In [12]:
delta_ra_arcseconds

array([  0. ,   0. , -19.8])

#### 4. Compute RA distance

In [13]:
mean_dec = np.mean([
    cyg_A_61_dec,
    cyg_B_61_dec
], axis=0)

In [14]:
mean_dec_radians = float(astro.to_radians(
    sf=3, 
    degs=mean_dec[0], 
    arcmins=mean_dec[1], 
    arcsecs=mean_dec[2]
    )
)

In [15]:
mean_dec_radians

0.676

Convert the delta in RA to the corresponding value given the declination.

In [16]:
distance_ra_arcsecs = \
    delta_ra_arcseconds * math.cos(mean_dec_radians)

In [17]:
distance_ra_arcsecs

array([  0.        ,   0.        , -15.44561694])

#### 5. Compute theta

Use Pythagoras theorem.

In [18]:
theta = math.sqrt(
    distance_ra_arcsecs[2]**2 + delta_dec[2]**2
)

In [19]:
astro.to_sf(sf=2, res=theta)

'31'

### Example 1.5

Converting angular separation to linear separation (or angular size to linear size).

Given the angular separation (or size) between two objects, we just need to know the distance to the objects to obtain the linear separation (or size).

This conversion relates the angular separation $\theta$ to the linear separation $D$ via the distance away $d$ using trigonometry:

$$\sin \theta = \frac{D}{d}$$

Taking the result of the previous example for the angular separation between _61 Cyg A_ and _61 Cyg B_, and the fact that they are 3.50 pc away from us, calculate their linear separation in AU.

First we can make use of the small angle approximation and use $\theta$ in radians as value for $\sin \theta$.

In [20]:
theta_radians = astro.to_radians(
    sf=6,
    arcsecs=theta
)

In [21]:

theta_radians

'0.000148706'

Then we can apply the formulate above.

In [22]:
linear_distance_pc = float(theta_radians) * 3.5

In [23]:
linear_distance_pc

0.000520471

In [24]:
astro.to_sf(
    sf=3, 
    res=astro.pc_to_au(linear_distance_pc))

'107'

### Question 1.7

Combining proper motion components.

The aparent movement of a star across the sky is called *proper motion*, and we can use the same approach used when computing angular and linear distances to determine movement.

Proper motion components refer to the proper motion in right ascension and declination, and can be defined as the changes in each component in time $\Delta t$.

Like in Example 1.4, we can use Pythagoras theorem to combine both components into the magnitude of the proper motions as:

$$\mu^2 = (\mu_{\alpha^*})^2 + (\mu_{\delta})^2$$

where $\mu_{\alpha^*}$ already includes the cosine factor of the declination.

Use the previous result to calculate the magnitude of the proper motion of Vega, knowing that

$$\mu_{\alpha^*} = +202 \text{ mas } y^{-1}$$
$$\mu_{\delta} = +287 \text{ mas } y^{-1}$$

What angular distance does the star move in 50 years?

In [25]:
vega_proper_motion_magnitude = float(astro.magnitude_from_components(
    sf=3,
    ra_comp=202,
    dec_comp=287
))

In [26]:
1e-3 * vega_proper_motion_magnitude * 50

17.55

### Example 1.6

##### Converting proper motion to transverse velocity.

The **transverse velocity**, $v_T$, of an astronomical object is to its proper motion what its linear separation or size is to its angular separation or size.

This transverse velocity is across the plane of the sky. There is also a radial component in the direction towards or away from use, which is not part of this transverse velocity. Both components, transverse and radial, make the **true velocity** of the astronomical object.

To calculate the transverse velocity, we need to know the magnitude of the proper motion, $\mu$, and the objects's distance, $d$. Then we use trigonometry the same way we do with linear separation and size:

$$\sin \mu = \frac{v_T}{d}$$


Using the value obtained above for the proper motion magitude for Vega, and knowing that the star is located 7.68 parsecs away, calculate irs transverse velocity in $km$ $s^{-1}$.

First we need to convert the proper motion pagnitude value to radians.

In [27]:
vega_proper_motion_magnitude_rads = float(astro.to_radians(
    sf=3,
    degs=0,
    arcmins=0,
    arcsecs=1e-3*float(vega_proper_motion_magnitude)
))

In [28]:
vega_proper_motion_magnitude_rads

1.7e-06

In [29]:
astro.angular_to_linear(
    sf=5,
    theta=vega_proper_motion_magnitude_rads / astro.SEC_IN_A_YEAR,
    distance_away=astro.KM_IN_A_PC * 7.68
)

'12.775'

### Question 1.8

##### From proper motion components to transverse velocity.


Calculate the transverse velocity of *Beta Virginis* which has proper motion components of $\mu_{\alpha^*} = +743 \text{ mas }y^{-1}$ and $\mu_{\delta} = -272 \text{ mas }y^{-1}$ and lies at a distance of 11.1 parsecs. Express your final answers to 3 significant figures.

The magnitude of the proper motion can be determined using Pythagoras theorem.

In [30]:
beta_virginis_pm_magnitude = float(astro.magnitude_from_components(
    sf=3,
    ra_comp=743,
    dec_comp=-272
))

In [31]:
beta_virginis_pm_magnitude

791.0

We can now use this magnitude and the distance to obtain the transverse velocity using trigonometry and the small angle approximation. First we need to convert the magnitude to radians.

In [35]:
beta_virginis_pm_magnitude_rad = float(astro.to_radians(
    sf=3,
    arcsecs=1e-3*beta_virginis_pm_magnitude
))

In [36]:
beta_virginis_pm_magnitude_rad

3.83e-06

In [38]:
float(astro.angular_to_linear(
    sf=3,
    theta=beta_virginis_pm_magnitude_rad / astro.SEC_IN_A_YEAR,
    distance_away=11.1*astro.KM_IN_A_PC
))

41.6

Try now with 78 Ursae Majoris which has proper motion components of $\mu_{\alpha^*} = +108 \text{ mas }y^{-1}$ and $\mu_{\delta} = +2 \text{ mas }y^{-1}$ and lies at a distance of 25.0 parsecs. Express your final answers to 3 significant figures.

In [39]:
ursae_majoris_78_pm_magnitude = float(astro.magnitude_from_components(
    sf=3,
    ra_comp=108,
    dec_comp=2
))

In [40]:
ursae_majoris_78_pm_magnitude

108.0

In [41]:
ursae_majoris_78_pm_magnitude_rad = float(astro.to_radians(
    sf=3,
    arcsecs=1e-3*ursae_majoris_78_pm_magnitude
))

In [42]:
ursae_majoris_78_pm_magnitude_rad

5.24e-07

In [43]:
float(astro.angular_to_linear(
    sf=3,
    theta=ursae_majoris_78_pm_magnitude_rad / astro.SEC_IN_A_YEAR,
    distance_away=25.0*astro.KM_IN_A_PC
))

12.8